# WESAD Validation Notebook for FLIRT


In [1]:
# Import Packages
import pandas as pd
import numpy as np

import matplotlib; matplotlib.use('agg')
import matplotlib.pyplot as plt

import multiprocessing
from joblib import Parallel, delayed
from tqdm.autonotebook import trange

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import utils, model_selection, metrics

from datetime import datetime, timedelta, timezone
from typing import List
import lightgbm as lgb
import glob2
import os 

import sys
sys.path.insert(1, '/home/fefespinola/ETHZ_Fall_2020/flirt-1')
import flirt.simple

In [2]:
The following function retrieves all HRV, EDA and ACC features per subject using the FLIRT pipeline


SyntaxError: invalid syntax (<ipython-input-2-c9c2e5945b79>, line 1)

In [2]:
def get_features_per_subject(path, window_length):
    features = flirt.simple.get_features_for_empatica_archive(zip_file_path = path,
                                      window_length = window_length,
                                      window_step_size = 0.25,
                                      hrv_features = False,
                                      eda_features = True,
                                      acc_features = False,
                                      bvp_features = False,
                                      temp_features = False,
                                      debug = True)
    return features

The following function determines the time offsets of the start and end of each relevant analysis period (baseline, stress, amusement). These offsets are combined with the timestamp stating the start of recording, to determine the absolute timestamps of the sections of interest for each subject. 

In [3]:
def find_label_timestamps(csv_path, StartingTime):

    ID = csv_path.split('/', 3)[2]
    df_timestamp = pd.read_csv(glob2.glob('project_data/WESAD/' + ID + '/*quest.csv')[0], delimiter = ';', header = 1).iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    print(df_timestamp.head())
    
    # Start/End of experiment periods
    print('\nStart of the baseline: ' + str(df_timestamp['Base'][0]))
    print('End of the baseline: ' + str(df_timestamp['Base'][1]))
    print('Start of the fun: ' + str(df_timestamp['Fun'][0]))
    print('End of the fun: ' + str(df_timestamp['Fun'][1]))
    print('Start of the stress: ' + str(df_timestamp['TSST'][0]))
    print('End of the stress: ' + str(df_timestamp['TSST'][1]))
    
    # Get start and end time and assign label into a dict
    lab_dict = {'Base':0, 'TSST':1, 'Fun':2}
    labels_times_dict = {}
    for mode in df_timestamp.columns.tolist():
        print('mode', mode)
        if mode=='Base' or mode=='Fun' or mode=='TSST':
            labels_times_dict[mode] = [StartingTime + timedelta(minutes = int(str(df_timestamp[mode][0]).split(".")[0]))+ timedelta                                         (seconds = int(str(df_timestamp[mode][0]).split(".")[1])), 
                                    StartingTime + timedelta(minutes = int(str(df_timestamp[mode][1]).split(".")[0])) + timedelta                                           (seconds = int(str(df_timestamp[mode][1]).split(".")[1])), lab_dict[mode]]
            
            #labels_times_dict[mode] = [StartingTime + timedelta(minutes = float(df_timestamp[mode][0])), 
                                  #StartingTime + timedelta(minutes = float(df_timestamp[mode][1])), lab_dict[mode]]
        
    return labels_times_dict

In [4]:
import ast
from dateutil.parser import parse

def find_label_start_time(csv_path):
    ID = csv_path.split('/', 3)[2]
    timestamp = open(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], "r")
    for i in range(2):
        line = (timestamp.readline())
        line = line.strip()[2:]
        if i==1:
            dict = ast.literal_eval(line)
            start_time_str = dict['00:07:80:D8:AB:58']['time']
            date_str = dict['00:07:80:D8:AB:58']['date']
            datetime_str = date_str + " " + start_time_str
            #print(datetime_str)
            date_time_obj = parse(datetime_str)
            #print(date_time_obj)
            start_time = date_time_obj
            utc_time = start_time - timedelta(hours=2)

    timestamp.close()

    #df_timestamp = pd.read_table(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], delim_whitespace=True)#.iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    #print(df_timestamp.head())
    return utc_time

Plots the training and validation classification metric evolution with the number of iterations. 

In [5]:
os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
for subject_path in File_Path:
    start_time = find_label_start_time(subject_path)
    print(start_time)

Printing the timestamp for S10
2017-07-25 07:25:01
Printing the timestamp for S11
2017-07-25 11:33:01
Printing the timestamp for S13
2017-08-08 11:33:01
Printing the timestamp for S14
2017-08-09 07:31:01
Printing the timestamp for S15
2017-08-10 07:30:01
Printing the timestamp for S16
2017-08-10 12:21:01
Printing the timestamp for S17
2017-08-11 07:39:01
Printing the timestamp for S2
2017-05-22 07:39:01
Printing the timestamp for S3
2017-05-24 11:26:01
Printing the timestamp for S4
2017-06-13 08:57:01
Printing the timestamp for S5
2017-06-13 12:42:01
Printing the timestamp for S6
2017-06-14 11:40:01
Printing the timestamp for S7
2017-07-06 11:30:01
Printing the timestamp for S8
2017-07-10 11:28:01
Printing the timestamp for S9
2017-07-11 11:26:01


In [6]:
def render_metric(eval_results, metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/render_metric_all_ekf_feat.png')

Plots the 10 top important classification features, i.e. the ones that influence the output the most.

In [7]:
def render_plot_importance(gbm, importance_type, max_features=10, ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/feature_importance_all_ekf_feat.png')


Main function that calls the above functions, determines the relevant data to use (i.e. that within the useful recording periods of baseline, stress and amusement) using the timestampp offsets, assignes the appropriate label to each sample and returns the full data with training samples and the corresponding labels.

In [8]:
def main():
    os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
    df_all = pd.DataFrame(None)
    #relevant_features = pd.DataFrame(None)
    File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
    window_length = 60 # in seconds
    window_shift = 0.25 # in seconds
    for subject_path in File_Path:
        print(subject_path)
        print(subject_path.split('/', 3)[2])
        ID = subject_path.split('/', 3)[2]
        zip_path = glob2.glob('project_data/WESAD/' + ID + '/*_Data.zip')[0]
        print(zip_path)
        features = get_features_per_subject(zip_path, window_length)
        features.index.name = 'timedata'
        E4Time = features.index[0]
        print(E4Time)
        StartingTime = find_label_start_time(subject_path)
        print(StartingTime)
        labels_times = find_label_timestamps(subject_path, StartingTime)
        #features.index.tz_localize(tz='UTC')
        relevant_features = features.loc[
            ((features.index.tz_localize(tz=None) >= labels_times['Base'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Base'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['Fun'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Fun'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['TSST'][0]) & (features.index.tz_localize(tz=None) <= labels_times['TSST'][1]))]

        relevant_features.insert(0, 'ID', ID)
        relevant_features['label'] = np.zeros(len(relevant_features))
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['Fun'][0]) &
                                (relevant_features.index.tz_localize(tz=None)<=labels_times['Fun'][1]), 'label'] = labels_times['Fun'][2]
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['TSST'][0]) & 
                            (relevant_features.index.tz_localize(tz=None)<=labels_times['TSST'][1]), 'label'] = labels_times['TSST'][2]

        # concatenate all subjects and add IDs
        df_all = pd.concat((df_all, relevant_features))
    
    print(df_all)

    return df_all

This function generates and saves feature matrices for the individual physiological signals 

In [9]:
def __get_subset_features(df_all, feature_name: str, eda_method:str='lpf'):
    
    if feature_name=='physio':
        small_df = df_all.loc[:, df_all.columns.str.startswith('hrv')&df_all.columns.str.startswith   ('eda')&df_all.columns.str.startswith('bvp')&df_all.columns.str.startswith('temp')]
        filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
    else:
        small_df = df_all.loc[:, df_all.columns.str.startswith(feature_name)]
        if feature_name=='eda':
            filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
        else:
            filename = 'features_all_' + features_name + '_feat.csv'
    small_df.to_csv(filename)


The following function retrieves the correct training and testing data for LOSO cross-validation. It also deals with missing data (inf and nan), and scales the features.

In [10]:
def __get_train_valid_data(df_all, cv_subject):
    scaler = StandardScaler()

    #training data
    X_train = df_all.loc[df_all['ID']!=cv_subject]  # 500 entities, each contains 10 features
    X_train = X_train.iloc[:, 1:len(df_all.columns)-1]
    X_train = X_train.replace([np.nan, np.inf, -np.inf], -1000)
    X_train = scaler.fit_transform(X_train)
    y_train = df_all.loc[df_all['ID']!=cv_subject, ['label']]  # binary target
    train_data = lgb.Dataset(X_train, label=y_train)

    #validation data
    X_test = df_all.loc[df_all['ID']==cv_subject]  # 500 entities, each contains 10 features
    X_test = X_test.iloc[:, 1:len(df_all.columns)-1]
    X_test = X_test.replace([np.nan, np.inf, -np.inf], -1000)
    X_test = scaler.transform(X_test)
    y_test = df_all.loc[df_all['ID']==cv_subject, ['label']]   # binary target
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    return X_train, y_train, train_data, X_test, y_test, test_data
 

Run the evaluation script to retrieve the labeled data and train classifier to output f1-score

In [11]:
if __name__ == '__main__':
    df_all = main()
    df_all.to_csv('/home/fefespinola/ETHZ_Fall_2020/features_all_eda_ekf_leda.csv')
    df_all = pd.read_csv('/home/fefespinola/ETHZ_Fall_2020/features_all_eda_ekf_leda.csv')
    df_all.set_index('timedata', inplace=True)
    #df_all = df_all.loc[:, ~df_all.columns.str.startswith('hrv')]
    print(df_all)
    ID=df_all.ID
    print(ID.unique())

    ### for binary classification uncomment line below 
    #df_all['label'].replace(2, 0, inplace=True)
    #print('===== BINARY ====')

    print('---start classification---')

    df_all.round(4)
    df = df_all.replace([np.inf, -np.inf], np.nan) # np.inf leads to problems with some techniques

    # Clean columns that contain a lot of nan values 
    print(len(df), len(df.columns))
    df = df.dropna(axis=1, thresh=int(len(df)*0.8))
    print(len(df), len(df.columns))
    print('Columns dropped: ', df_all.drop(df.columns, axis=1).columns.values)

    stats = []

    cv = model_selection.LeaveOneGroupOut()

    X = df.drop(columns=['label', 'ID'])
    y = df['label'].astype('int')
    groups = df['ID']
    print("running %d-fold CV..." % (cv.get_n_splits(X, y, groups)))

    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots(figsize=(15, 15))

    for train_index, test_index in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #### for binary classification uncomment line below
        #params = {'objective': 'binary', 'is_unbalance': True}
        params = {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class':3,  'is_unbalance': True}
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        stats.append({
            'f1': metrics.f1_score(y_test, y_pred, average="macro")})
            
        #print(metrics.classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

    stats = pd.DataFrame(stats)
    print(stats.f1.mean())

    '''
    #parameters
    param = {'metric': 'auc_mu', 'learning_rate': 0.01, 'num_leaves': 31, 'is_unbalance':True,
        'verbose': 1, 'objective':'multiclass', 'num_class':3, 'lambda_l1':0, 'force_col_wise':True}
    
    subjects = ['S2', 'S3', 'S4', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']

    ##### Start Classification
    f1_tot = 0
    f1_dict = {}
    #data with validation set
    for subj in subjects:
        # get data
        print('===Training for LOSO ', subj, '===')
        _, _, train_data, X_test, y_test, test_data = __get_train_valid_data(df_all, subj)
    
        evals_result = {}  # to record eval results for plotting
        
        #train normally
        bst_norm = lgb.train(param, train_data, num_boost_round=550, valid_sets=[train_data, test_data],                        evals_result=evals_result, verbose_eval=10)

        best_iteration = np.argmax(evals_result['valid_1']['auc_mu'][2:]) + 1
        print ('best iter', best_iteration)
        y_pred = bst_norm.predict(X_test, num_iteration=best_iteration)
        y_pred = np.argmax(y_pred, axis=1)

        f1_metric = f1_score(y_test, y_pred, average='macro')
        f1_dict[subj] = f1_metric
        print(f1_dict)
        f1_tot = f1_tot + f1_metric
        print(confusion_matrix(y_test, y_pred))


    f1_tot = f1_tot/len(subjects)
    print(f1_tot)
    #render_metric(evals_result, param['metric'])
    #render_plot_importance(bst_norm, importance_type='split')
    '''

project_data/WESAD/S10/S10_readme.txt
S10
project_data/WESAD/S10/S10_E4_Data.zip
Reading files
Calculating EDA features



                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-07-25 07:06:08+00:00            0.028666    0.035221    0.000010   
2017-07-25 07:06:08.250000+00:00     0.028688    0.035205    0.000018   
2017-07-25 07:06:08.500000+00:00     0.028710    0.035189    0.001026   
2017-07-25 07:06:08.750000+00:00     0.028730    0.035175    0.003667   
2017-07-25 07:06:09+00:00            0.028738    0.035169    0.003667   
...                                       ...         ...         ...   
2017-07-25 08:59:48.500000+00:00     0.016628    0.000911    0.015331   
2017-07-25 08:59:48.750000+00:00     0.016887    0.000770    0.015787   
2017-07-25 08:59:49+00:00            0.017162    0.000602    0.016329   
2017-07-25 08:59:49.250000+00:00     0.017439    0.000419    0.016916   
2017-07-25 08:59:49.500000+00:00     0.017701    0.000242    0.017459   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-07-25 11:15:19+00:00            0.324642    0.238810    0.000092   
2017-07-25 11:15:19.250000+00:00     0.328192    0.240288    0.000253   
2017-07-25 11:15:19.500000+00:00     0.331881    0.242014    0.000365   
2017-07-25 11:15:19.750000+00:00     0.335728    0.244045    0.000384   
2017-07-25 11:15:20+00:00            0.339752    0.246432    0.000384   
...                                       ...         ...         ...   
2017-07-25 13:02:58+00:00            0.023916    0.003654    0.018869   
2017-07-25 13:02:58.250000+00:00     0.024925    0.003148    0.020657   
2017-07-25 13:02:58.500000+00:00     0.025992    0.002587    0.022584   
2017-07-25 13:02:58.750000+00:00     0.027128    0.001940    0.024769   
2017-07-25 13:02:59+00:00            0.028308    0.001211    0.027097   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-08-08 11:14:07+00:00            0.304327    0.230378    0.000097   
2017-08-08 11:14:07.250000+00:00     0.306620    0.230077    0.000248   
2017-08-08 11:14:07.500000+00:00     0.308776    0.229620    0.000275   
2017-08-08 11:14:07.750000+00:00     0.310793    0.229027    0.001346   
2017-08-08 11:14:08+00:00            0.312667    0.228326    0.005520   
...                                       ...         ...         ...   
2017-08-08 13:08:28+00:00            0.086860    0.015123    0.070022   
2017-08-08 13:08:28.250000+00:00     0.081610    0.010444    0.070022   
2017-08-08 13:08:28.500000+00:00     0.077289    0.006556    0.070022   
2017-08-08 13:08:28.750000+00:00     0.073970    0.003637    0.070022   
2017-08-08 13:08:29+00:00            0.071554    0.001532    0.070022   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-08-09 07:10:31+00:00            0.042593    0.031618    0.000008   
2017-08-09 07:10:31.250000+00:00     0.042722    0.031507    0.000015   
2017-08-09 07:10:31.500000+00:00     0.042846    0.031397    0.001111   
2017-08-09 07:10:31.750000+00:00     0.042962    0.031293    0.001324   
2017-08-09 07:10:32+00:00            0.043062    0.031207    0.001324   
...                                       ...         ...         ...   
2017-08-09 09:06:55+00:00            0.004738    0.001281    0.003109   
2017-08-09 09:06:55.250000+00:00     0.005064    0.001154    0.003363   
2017-08-09 09:06:55.500000+00:00     0.005489    0.000872    0.004223   
2017-08-09 09:06:55.750000+00:00     0.005912    0.000549    0.005219   
2017-08-09 09:06:56+00:00            0.006258    0.000304    0.005954   

                                  phasic_max  pha

...


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-08-10 12:00:25+00:00            0.032490    0.037154    0.000005   
2017-08-10 12:00:25.250000+00:00     0.032508    0.037140    0.000010   
2017-08-10 12:00:25.500000+00:00     0.032525    0.037126    0.001027   
2017-08-10 12:00:25.750000+00:00     0.032539    0.037115    0.002597   
2017-08-10 12:00:26+00:00            0.032540    0.037114    0.002597   
...                                       ...         ...         ...   
2017-08-10 13:58:50.500000+00:00     0.021271    0.004324    0.014590   
2017-08-10 13:58:50.750000+00:00     0.022607    0.003425    0.017413   
2017-08-10 13:58:51+00:00            0.023905    0.002496    0.020429   
2017-08-10 13:58:51.250000+00:00     0.025064    0.001714    0.022936   
2017-08-10 13:58:51.500000+00:00     0.026128    0.001004    0.025124   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-08-11 07:20:22+00:00            0.288616    0.261035    0.006830   
2017-08-11 07:20:22.250000+00:00     0.289342    0.260492    0.019209   
2017-08-11 07:20:22.500000+00:00     0.290034    0.259993    0.029158   
2017-08-11 07:20:22.750000+00:00     0.290701    0.259528    0.034316   
2017-08-11 07:20:23+00:00            0.291348    0.259088    0.034316   
...                                       ...         ...         ...   
2017-08-11 09:20:52+00:00            0.029360    0.025312    0.007641   
2017-08-11 09:20:52.250000+00:00     0.033704    0.025605    0.007983   
2017-08-11 09:20:52.500000+00:00     0.040134    0.024755    0.011768   
2017-08-11 09:20:52.750000+00:00     0.049590    0.021435    0.024632   
2017-08-11 09:20:53+00:00            0.062069    0.014901    0.047167   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-05-22 07:16:25+00:00            0.132067    0.063760    0.000031   
2017-05-22 07:16:25.250000+00:00     0.132892    0.063326    0.000079   
2017-05-22 07:16:25.500000+00:00     0.133727    0.062889    0.000088   
2017-05-22 07:16:25.750000+00:00     0.134577    0.062454    0.001313   
2017-05-22 07:16:26+00:00            0.135430    0.062025    0.004804   
...                                       ...         ...         ...   
2017-05-22 09:27:37+00:00            0.011017    0.002152    0.007249   
2017-05-22 09:27:37.250000+00:00     0.010620    0.002146    0.007249   
2017-05-22 09:27:37.500000+00:00     0.010002    0.001963    0.007249   
2017-05-22 09:27:37.750000+00:00     0.009188    0.001576    0.007249   
2017-05-22 09:27:38+00:00            0.008227    0.000978    0.007249   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-05-24 11:09:48+00:00            0.488131    0.243504    0.003796   
2017-05-24 11:09:48.250000+00:00     0.490263    0.241485    0.010669   
2017-05-24 11:09:48.500000+00:00     0.492311    0.239485    0.016163   
2017-05-24 11:09:48.750000+00:00     0.494275    0.237497    0.019805   
2017-05-24 11:09:49+00:00            0.496155    0.235515    0.019805   
...                                       ...         ...         ...   
2017-05-24 13:18:30+00:00            0.008583    0.000061    0.008485   
2017-05-24 13:18:30.250000+00:00     0.008564    0.000047    0.008485   
2017-05-24 13:18:30.500000+00:00     0.008548    0.000039    0.008485   
2017-05-24 13:18:30.750000+00:00     0.008535    0.000037    0.008485   
2017-05-24 13:18:31+00:00            0.008515    0.000030    0.008485   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-06-13 08:34:40+00:00            0.016802    0.017391    0.000005   
2017-06-13 08:34:40.250000+00:00     0.016811    0.017383    0.000009   
2017-06-13 08:34:40.500000+00:00     0.016821    0.017374    0.000965   
2017-06-13 08:34:40.750000+00:00     0.016827    0.017368    0.002186   
2017-06-13 08:34:41+00:00            0.016821    0.017373    0.002186   
...                                       ...         ...         ...   
2017-06-13 10:47:58+00:00            0.029022    0.000602    0.027981   
2017-06-13 10:47:58.250000+00:00     0.028873    0.000550    0.027981   
2017-06-13 10:47:58.500000+00:00     0.028711    0.000496    0.027981   
2017-06-13 10:47:58.750000+00:00     0.028516    0.000420    0.027981   
2017-06-13 10:47:59+00:00            0.028271    0.000290    0.027981   

                                  phasic_max  pha

# Get relevant features
